<a href="https://colab.research.google.com/github/kashafali8/Deep_Reinforcement_Learning_Recommenders/blob/main/DRL_Recommenders/Dataset_RR/SNQN_RR_ItemFeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!rm -R /content/Deep_Reinforcement_Learning_Recommenders
!git clone https://github.com/kashafali8/Deep_Reinforcement_Learning_Recommenders.git

rm: cannot remove '/content/Deep_Reinforcement_Learning_Recommenders': No such file or directory
Cloning into 'Deep_Reinforcement_Learning_Recommenders'...
remote: Enumerating objects: 437, done.
remote: Counting objects: 100% (187/187), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 437 (delta 123), reused 115 (delta 59), pack-reused 250
Receiving objects: 100% (437/437), 8.49 MiB | 17.11 MiB/s, done.
Resolving deltas: 100% (235/235), done.


In [ ]:
!pip install trfl
!pip install kaggle

In [16]:
!mv /content/drive/MyDrive/kaggle.json /root/.kaggle/kaggle.json

In [17]:
!kaggle datasets download -d retailrocket/ecommerce-dataset
!unzip ecommerce-dataset.zip -d Deep_Reinforcement_Learning_Recommenders/DRL_Recommenders/Dataset_RR/data/
!rm ecommerce-dataset.zip

100% 291M/291M [00:14<00:00, 23.3MB/s]
100% 291M/291M [00:14<00:00, 21.2MB/s]
Archive:  ecommerce-dataset.zip
  inflating: Deep_Reinforcement_Learning_Recommenders/DRL_Recommenders/Dataset_RR/data/category_tree.csv  
  inflating: Deep_Reinforcement_Learning_Recommenders/DRL_Recommenders/Dataset_RR/data/events.csv  
  inflating: Deep_Reinforcement_Learning_Recommenders/DRL_Recommenders/Dataset_RR/data/item_properties_part1.csv  
  inflating: Deep_Reinforcement_Learning_Recommenders/DRL_Recommenders/Dataset_RR/data/item_properties_part2.csv  


In [18]:
!python /content/Deep_Reinforcement_Learning_Recommenders/DRL_Recommenders/Dataset_RR/src/replay_buffer.py --data /content/Deep_Reinforcement_Learning_Recommenders/DRL_Recommenders/Dataset_RR/data/


Starting to pre-process data...

Sorting and pickling data...

Splitting data into train, validation, and test sets...

Pickling train, validation, and test sets...

Calculating item popularity and storing as dictionary...

Generating replay buffer from train set...

Pickling replay buffer...

Pickling data statistics...

Script completed successfully!


In [19]:
import pandas as pd

In [25]:
ip_df = pd.read_csv("/content/Deep_Reinforcement_Learning_Recommenders/DRL_Recommenders/Dataset_RR/data/item_properties_part1.csv")

In [26]:
ip_df

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513
...,...,...,...,...
10999994,1439694000000,86599,categoryid,618
10999995,1435460400000,153032,1066,n1020.000 424566
10999996,1440298800000,421788,888,35975 856003 37346
10999997,1437879600000,159792,400,n552.000 639502 n720.000 424566


In [31]:
import pandas as pd
import numpy as np


def create_feature_matrix(
    sorted_events, n_files=2, path_name="", one_hot_encode=True, top_features=500
):
    for i in range(n_files):
        if i == 0:
            item_features = pd.read_csv(path_name + str(i + 1) + ".csv")
        else:
            item_features = pd.concat(
                [item_features, pd.read_csv(path_name + str(i + 1) + ".csv")],
                ignore_index=True,
            )

    item_features = item_features[
        item_features["itemid"].isin(sorted_events["item_id"].unique().tolist())
    ].drop_duplicates()
    item_features["property_value"] = (
        item_features["property"].str.strip() + item_features["value"].str.strip()
    )
    item_features = item_features.drop(["timestamp"], axis=1).drop_duplicates()

    if one_hot_encode:
        one_hot_encoded = pd.DataFrame()
        itemids = []

        event_item_list = sorted_events.item_id.unique()
        event_item_list.sort()
        item_list = item_features["itemid"].unique()
        properties = (
            item_features["property_value"]
            .value_counts()
            .head(top_features)
            .index.tolist()
        )

        for item in event_item_list:
            if len(itemids) % 1000 == 0:
                print("hi")
            if item not in item_list:
                one_hot_encoded = pd.concat(
                    [one_hot_encoded, pd.DataFrame(np.zeros(len(properties))).T],
                    ignore_index=True,
                )
                itemids.append(item)
                continue

            item_properties = item_features[item_features["itemid"] == item][
                "property_value"
            ].unique()
            one_hot_encoded = pd.concat(
                [
                    one_hot_encoded,
                    pd.DataFrame(
                        [1 if x in item_properties else 0 for x in properties]
                    ).T,
                ],
                ignore_index=True,
            )
            itemids.append(item)

        return one_hot_encoded, itemids

    else:
        return item_features, item_features["itemid"].unique().tolist()

In [ ]:
sorted_events = pd.read_pickle("/content/Deep_Reinforcement_Learning_Recommenders/DRL_Recommenders/Dataset_RR/data/sorted_events.df")
one_hot_encoded, itemids = create_feature_matrix(
    sorted_events, n_files=2, path_name="/content/Deep_Reinforcement_Learning_Recommenders/DRL_Recommenders/Dataset_RR/data/item_properties_part", one_hot_encode=True
)

In [34]:
one_hot_encoded.to_pickle("/content/Deep_Reinforcement_Learning_Recommenders/DRL_Recommenders/Dataset_RR/data/one_hot_encoded.df")

In [38]:
!python "/content/Deep_Reinforcement_Learning_Recommenders/DRL_Recommenders/Dataset_RR/src/SNQN_IF.py" --model=SASRec --epoch=10 --data="/content/Deep_Reinforcement_Learning_Recommenders/DRL_Recommenders/Dataset_RR/data/"

2023-05-04 03:21:03.050822: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-04 03:21:03.103901: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 03:21:04.124652: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/Deep_Reinforcement_Learning_Recommenders/DRL_Recommenders/Dataset_RR/src/SNQN_IF.py:185: UserWarning: `tf.layers.dropout` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dropout` instead.
  self.seq = tf.co